<a href="https://colab.research.google.com/github/Kim-TaeKyoung/a/blob/main/1_2_EfficentDet_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 환경세팅(의존성 패키지 설치)

In [ ]:
import os
import sys
import random
import json
import time
import datetime
from IPython.display import Image, display

# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 패키지 설치
!cd /content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet && cat requirements_colab.txt | xargs -n 1 -L 1 pip install
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
!pip install efficientnet_pytorch
!pip install tensorboardX

sys.path.append("/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/");
from train_detector import Detector
from infer_detector import Infer

Mounted at /content/drive
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=dea9297311a2423b1ff9b5ebcce160385717d53ac8cd279c6727485128bfd04b
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
     |████████████████████████████████| 352.6 MB 5.9 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 124 kB 14.8 MB/s 
  Cloning https://github.com/abhi-kumar/cocoapi.git to /tmp/pip-install-trgapxqm/pycocotools_801aa525658f4b70859cc1872bc2ecee
  Running command git clone -q https://github.com/abhi-kumar/cocoapi.git /tmp/pip-install-trgapxqm/pycocotools_801aa525658f4b70859cc1872bc2ecee
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263846 s

In [ ]:
# Augmentation이 적용된 데이터셋 다운로드
!curl -L "https://app.roboflow.com/ds/D6ipNcT3p5?key=dPqULorgrb" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

!mkdir Fire
!mkdir Fire/annotations
!mkdir Fire/Images
%cp train/_annotations.coco.json Fire/annotations/instances_Images.json
%cp train/*.jpg Fire/Images/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: train/-135-60---fire_jpg.rf.b8cde97026e6de5dff355ab7b25fbb0f.jpg  
 extracting: train/-135-60---fire_jpg.rf.ebc90075e95d1b2e22f4152380dd33f7.jpg  
 extracting: train/-135-61---fire_jpg.rf.40cd67dc1be056f8aca505fbd794b35b.jpg  
 extracting: train/-135-61---fire_jpg.rf.a418fd9bffe2ab8819b5b7664a550c28.jpg  
 extracting: train/-135-61---fire_jpg.rf.fc68bf1d229bed31ef2d579f3cbcab90.jpg  
 extracting: train/-135-62---fire_jpg.rf.12c48803bd533fe39bcfbebdf3961b87.jpg  
 extracting: train/-135-62---fire_jpg.rf.d82de914464370a1c5351c7662c0598e.jpg  
 extracting: train/-135-62---fire_jpg.rf.f4fa3f77e3c1d5dde400d0184eeba54f.jpg  
 extracting: train/-135-63---fire_jpg.rf.16a32d32c911355822d58d3da39b07a1.jpg  
 extracting: train/-135-63---fire_jpg.rf.afbe70e38747ce686ee49c89054b32a9.jpg  
 extracting: train/-135-63---fire_jpg.rf.e94eadca65e553b21400db2291ae7b39.jpg  
 extracting: train/-135-66---fire_jpg.rf.518d4bba4a332fdf4b65fc85f301d99d.jpg  
 ext

# 2. 데이터셋 및 GPU 설정

In [ ]:
# 모델선언
aics = Detector()

root_dir = "./";
coco_dir = "Fire";
img_dir = "./";
set_dir = "Images";

# 데이터셋 정의
aics.Train_Dataset(root_dir, coco_dir, img_dir, set_dir, batch_size=16, image_size=512, use_gpu=True)

GPU
1
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


# 3. 훈련

In [ ]:
def weight_reset(m):
    reset_parameters = getattr(m, "reset_parameters", None)
    if callable(reset_parameters):
        m.reset_parameters()

In [ ]:
#모델 불러오기(Efficinet-b0)
aics.Model()

#옵티마이저 셋팅, Optimizer는 Adam, learning rate는 0.01, 3 epochs 마다 loss drop이 없을경우 learning_rate는 lr * 0.1로 감소, Early Stop은 껐음
aics.Set_Hyperparams(lr=0.0001, val_interval=10, es_min_delta=0.0, es_patience=0)
aics.system_dict['local']['model'].apply(weight_reset)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


DataParallel(
  (module): EfficientDet(
    (conv3): Conv2d(40, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv4): Conv2d(80, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv5): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv6): Conv2d(192, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv7): Sequential(
      (0): ReLU()
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bifpn): Sequential(
      (0): BiFPN(
        (conv6_up): ConvBlock(
          (conv): Sequential(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
            (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (2): BatchNorm2d(64, eps=4e-05, momentum=0.9997, affine=True, track_running_stats=True)
            (3): ReLU()
          )
        )
        (conv5_up): ConvBlock(
          (conv): Sequential(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64

In [ ]:
birthdate = datetime.datetime.now(datetime.timezone.utc)
weight_save_path = birthdate.astimezone().strftime('%Y-%m-%d %H:%M:%S %Z')
print(weight_save_path)

2021-10-22 01:05:01 UTC


In [ ]:
%%time

aics.Train(num_epochs=500, model_output_dir="/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/weight/"+ weight_save_path);

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/src/model.py:251: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  if len(inputs) == 2:
/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/src/utils.py:84: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  image_shape = np.array(image_shape)
/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/src/utils.py:96: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you cal

faild onnx export


/usr/local/lib/python3.7/dist-packages/torch/onnx/symbolic_opset9.py:701: UserWarning: This model contains a squeeze operation on dimension 1 on an input with unknown shape. Note that if the size of dimension 1 of the input is not 1, the ONNX model will return an error. Opset version 11 supports squeezing on non-singleton dimensions, it is recommended to export this model using opset version 11 or higher.
  "version 11 or higher.")



Epoch   109: reducing learning rate of group 0 to 1.0000e-05.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./tensorboard/signatrix_efficientdet_coco/

In [ ]:
# Validation 이미지 결과 확인

aics = Detector()
aics = Infer();
aics.Model(model_dir="/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/weight/2021-10-19 09:45:22 UTC")

class_list = ['L','fire', 'smoke']

import glob
from PIL import Image as PILImage
from matplotlib import pyplot as plt
from IPython.display import Image, clear_output, display
from IPython.core.magic import register_line_cell_magic
from natsort import natsorted

plt.rcParams.update({'figure.max_open_warning': 0})

test_data_dir = '/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/test-frame_rearranged/*.jpg'
test_images = natsorted(glob.glob(test_data_dir))


for fig in test_images:
  aics.Predict(fig, class_list, vis_threshold=0.5);
  display(Image(filename='output.jpg'))
  print("\n")
  

In [ ]:
import mAP_evaluation
import importlib
import torch

importlib.reload(mAP_evaluation)

# 모델선언
aics = Detector()

root_dir = "./";
coco_dir = "Fire_Valid";
img_dir = "./";
set_dir = "Images";

!mkdir -p Fire_Valid/annotations
!mkdir -p Fire_Valid/Images
%cp valid/_annotations.coco.json Fire_Valid/annotations/instances_Images.json
%cp valid/*.jpg Fire_Valid/Images/

aics.Val_Dataset(root_dir, coco_dir, img_dir, set_dir)

#모델 불러오기(Efficinet-b0)
model = torch.load("/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/weight/2021-09-30_17_02_38/signatrix_efficientdet_coco.pth")
val_cocoset = aics.system_dict["local"]["val_set"]

model.cuda()

mAP_evaluation.evaluate_coco(val_cocoset, model)